## Table of contents
* [Introduction: Business Problem](#introduction)
* [Data](#data)
* [Methodology](#methodology)
* [Analysis](#analysis)
* [Results and Discussion](#results)
* [Conclusion](#conclusion)



## Introduction: Business Problem <a name="introduction"></a>

The objective of this project is to find an optimal location for a restaurant. Specifically, this report will be targeted to stakeholders interested in opening an **Indian restaurant** in **Manhattan, New York City**.

Since there are lots of restaurants in Manahattan we will try to detect locations that are not already very crowded with restaurants. We are also particularly interested in areas with no Indian restaurants in vicinity. 

We will use our data science to generate a few most promissing neighborhoods based on this criteria. Advantages of each area will then be clearly expressed so that best possible final location can be chosen by stakeholders.

## Data <a name="data"></a>

Based on definition of our problem, factors that will influence our decission are:
* number of existing restaurants in the neighborhood (any type of restaurant)
* number of and distance to Indian restaurants in the neighborhood, if any

We decided to use regularly spaced grid of locations, centered around Times Square which is like heart of Manhattan NYC, to define our neighborhoods.

Following data sources will be needed to extract/generate the required information:
* centers of candidate areas will be generated algorithmically and approximate addresses of centers of those areas will be obtained using **Google Maps API reverse geocoding**
* number of restaurants and their type and location in every neighborhood will be obtained using **Foursquare API**

### Neighborhood Candidates

Let's create latitude & longitude coordinates for centroids of our candidate neighborhoods. We will create a grid of cells covering our area of interest which is aprox. 12x12 kilometers centered around Times Square.

Let's first find the latitude & longitude of Times Square, using specific, well known address and Google Maps geocoding API.

In [1]:
# API key is hidden
google_api_key = "xxxxx"

In [2]:
import requests

def get_coordinates(api_key, address, verbose=False):
    try:
        url = 'https://maps.googleapis.com/maps/api/geocode/json?key={}&address={}'.format(api_key, address)
        response = requests.get(url).json()
        if verbose:
            print('Google Maps API JSON result =>', response)
        results = response['results']
        geographical_data = results[0]['geometry']['location'] # get geographical coordinates
        lat = geographical_data['lat']
        lon = geographical_data['lng']
        return [lat, lon]
    except:
        return [None, None]
    
address = 'Times Square, New York City, NY, USA'
Times_Square = get_coordinates(google_api_key, address)
print('Coordinate of {}: {}'.format(address, Times_Square))

Coordinate of Times Square, New York City, NY, USA: [40.7579747, -73.9855426]


Now let's create a grid of area candidates, equaly spaced, centered around Times Square and within ~4km from it. Our neighborhoods will be defined as circular areas with a radius of 300 meters, so our neighborhood centers will be 600 meters apart.

To accurately calculate distances we need to create our grid of locations in Cartesian 2D coordinate system which allows us to calculate distances in meters (not in latitude/longitude degrees). Then we'll project those coordinates back to latitude/longitude degrees to be shown on Folium map. So let's create functions to convert between WGS84 spherical coordinate system (latitude/longitude degrees) and UTM Cartesian coordinate system (X/Y coordinates in  meters).

In [3]:
!pip install shapely
import shapely.geometry

!pip install pyproj
import pyproj

import math

def lonlat_to_xy(lon, lat):
    proj_latlon = pyproj.Proj(proj='latlong',datum='WGS84')
    proj_xy = pyproj.Proj(proj="utm", zone=33, datum='WGS84')
    xy = pyproj.transform(proj_latlon, proj_xy, lon, lat)
    return xy[0], xy[1]

def xy_to_lonlat(x, y):
    proj_latlon = pyproj.Proj(proj='latlong',datum='WGS84')
    proj_xy = pyproj.Proj(proj="utm", zone=33, datum='WGS84')
    lonlat = pyproj.transform(proj_xy, proj_latlon, x, y)
    return lonlat[0], lonlat[1]

def calc_xy_distance(x1, y1, x2, y2):
    dx = x2 - x1
    dy = y2 - y1
    return math.sqrt(dx*dx + dy*dy)

print('Coordinate transformation check')
print('-------------------------------')
print('Times Square longitude={}, latitude={}'.format(Times_Square[1], Times_Square[0]))
x, y = lonlat_to_xy(Times_Square[1], Times_Square[0])
print('Times Square UTM X={}, Y={}'.format(x, y))
lo, la = xy_to_lonlat(x, y)
print('Times Square longitude={}, latitude={}'.format(lo, la))

     |████████████████████████████████| 1.0MB 6.1MB/s eta 0:00:01
     |████████████████████████████████| 10.9MB 6.2MB/s eta 0:00:01
Coordinate transformation check
-------------------------------
Times Square longitude=-73.9855426, latitude=40.7579747
Times Square UTM X=-5814547.571920981, Y=9867169.10935682
Times Square longitude=-73.98554259999962, latitude=40.757974699998876


/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:12: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:18: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1


Let's create a **hexagonal grid of cells**: we offset every other row, and adjust vertical row spacing so that **every cell center is equally distant from all it's neighbors**.

In [4]:
Times_Square_x, Times_Square_y = lonlat_to_xy(Times_Square[1], Times_Square[0]) # City center in Cartesian coordinates

k = math.sqrt(3) / 2 # Vertical offset for hexagonal grid cells
x_min = Times_Square_x - 4000
x_step = 600
y_min = Times_Square_y - 4000 - (int(21/k)*k*600 - 8000)/2
y_step = 600 * k 

latitudes = []
longitudes = []
distances_from_center = []
xs = []
ys = []
for i in range(0, int(21/k)):
    y = y_min + i * y_step
    x_offset = 300 if i%2==0 else 0
    for j in range(0, 21):
        x = x_min + j * x_step + x_offset
        distance_from_center = calc_xy_distance(Times_Square_x, Times_Square_y, x, y)
        if (distance_from_center <= 4001):
            lon, lat = xy_to_lonlat(x, y)
            latitudes.append(lat)
            longitudes.append(lon)
            distances_from_center.append(distance_from_center)
            xs.append(x)
            ys.append(y)

print(len(latitudes), 'candidate neighborhood centers generated.')

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:12: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:18: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:18: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:18: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:18: DeprecationWarning: This function

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:18: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:18: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:18: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:18: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:18: DeprecationWarning: This function

161 candidate neighborhood centers generated.


/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:18: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:18: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:18: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:18: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:18: DeprecationWarning: This function

Let's visualize the data we have so far: Times Square location and candidate neighborhood centers:

In [5]:
!pip install folium

import folium

     |████████████████████████████████| 102kB 5.3MB/s ta 0:00:011


In [6]:
map_NYC = folium.Map(location=Times_Square, zoom_start=13)
folium.Marker(Times_Square, popup='Times Square').add_to(map_NYC)
for lat, lon in zip(latitudes, longitudes):
    #folium.CircleMarker([lat, lon], radius=2, color='blue', fill=True, fill_color='blue', fill_opacity=1).add_to(map_berlin) 
    folium.Circle([lat, lon], radius=300, color='blue', fill=False).add_to(map_NYC)
    #folium.Marker([lat, lon]).add_to(map_NYC)
map_NYC

OK, we now have the coordinates of centers of neighborhoods/areas to be evaluated, equally spaced (distance from every point to it's neighbors is exactly the same) and within ~4km from Times Square. 

Let's now use Google Maps API to get approximate addresses of those locations.

In [7]:

def get_address(api_key, latitude, longitude, verbose=False):
    try:
        url = 'https://maps.googleapis.com/maps/api/geocode/json?key={}&latlng={},{}'.format(api_key, latitude, longitude)
        response = requests.get(url).json()
        if verbose:
            print('Google Maps API JSON result =>', response)
        results = response['results']
        address = results[0]['formatted_address']
        return address
    except:
        return None

addr = get_address(google_api_key, Times_Square[0], Times_Square[1])
print('Reverse geocoding check')
print('-----------------------')
print('Address of [{}, {}] is: {}'.format(Times_Square[0], Times_Square[1], addr))

Reverse geocoding check
-----------------------
Address of [40.7579747, -73.9855426] is: Times Square, New York, NY 10036, USA


In [8]:
print('Obtaining location addresses: ', end='')
addresses = []
for lat, lon in zip(latitudes, longitudes):
    address = get_address(google_api_key, lat, lon)
    if address is None:
        address = 'NO ADDRESS'
    address = address.replace(', USA', '') # We don't need country part of address
    addresses.append(address)
    print(' .', end='')
print(' done.')

Obtaining location addresses:  . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . done.


In [9]:
addresses[10:20]

['14 Sutton Pl S, New York, NY 10022',
 '1102 1st Avenue, New York, NY 10065',
 '310 E 65th St, New York, NY 10065',
 '219b E 69th St, New York, NY 10021',
 '165 E 73rd St, New York, NY 10021',
 '57th Ave, Long Island City, NY 11101',
 'Queens Midtown Tunnel, New York, NY',
 '229 43rd St, New York, NY 10036',
 '438 E 48th St, New York, NY 10017',
 '355 E 52nd St, New York, NY 10022']

Looking good. Let's now place all this into a Pandas dataframe.

In [10]:
import pandas as pd

df_locations = pd.DataFrame({'Address': addresses,
                             'Latitude': latitudes,
                             'Longitude': longitudes,
                             'X': xs,
                             'Y': ys,
                             'Distance from center': distances_from_center})

df_locations.head(10)

,Address,Latitude,Longitude,X,Y,Distance from center
0,"Long Island City, Center Blvd &, 46th Ave, New...",40.747956,-73.957686,-5.816148e+06,9.863532e+06,3973.663297
1,"Southpoint Park, Roosevelt Island, NY 10044",40.751493,-73.957557,-5.815548e+06,9.863532e+06,3772.267223
2,"200 W Loop Rd, New York, NY 10044",40.755030,-73.957428,-5.814948e+06,9.863532e+06,3659.234893
3,"31 Ed Koch Queensboro Bridge Path, New York, N...",40.758567,-73.957299,-5.814348e+06,9.863532e+06,3642.801120
4,"1194 York Ave, New York, NY 10065",40.762105,-73.957170,-5.813748e+06,9.863532e+06,3724.244890
5,"404 E 69th St, New York, NY 10021",40.765642,-73.957041,-5.813148e+06,9.863532e+06,3897.435054
6,"1-01-1-02 54th Ave, Long Island City, NY 11101",40.742736,-73.961906,-5.817048e+06,9.864051e+06,3996.248241
7,"4749 Center Blvd, Long Island City, NY 11109",40.746272,-73.961777,-5.816448e+06,9.864051e+06,3651.027253
8,"E Rd, New York, NY 10044",40.749809,-73.961649,-5.815848e+06,9.864051e+06,3377.869151
9,"1025 E 51st Street Pedestrian Crossing, New Yo...",40.753346,-73.961520,-5.815248e+06,9.864051e+06,3195.309062


...and let's now save/persist this data into local file.

In [11]:
df_locations.to_pickle('./locations.pkl')    

### Foursquare
Now that we have our location candidates, let's use Foursquare API to get info on restaurants in each neighborhood.

We're interested in venues in 'food' category, but only those that are proper restaurants - coffe shops, pizza places, bakeries etc. are not direct competitors so we don't care about those. So we will include in out list only venues that have 'restaurant' in category name, and we'll make sure to detect and include all the subcategories of specific 'Indian restaurant' category, as we need info on Indian restaurants in the neighborhood.

In [12]:
# The code was removed by Watson Studio for sharing.
foursquare_client_id = 'xxxxx' # your Foursquare ID
foursquare_client_secret = 'xxxxx' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + foursquare_client_id)
print('CLIENT_SECRET:' + foursquare_client_secret)

Your credentails:
CLIENT_ID: T25GLMJMOIT0O3ODVN3DQR430QABXUCTQ0XPTPZ22VMCDK4X
CLIENT_SECRET:YZOH2JPZQ1RSKPG43P1LGN2DLTGBGTUFKKNO1I1ONDZT2ASF


In [13]:
# Category IDs corresponding to Italian restaurants were taken from Foursquare web site (https://developer.foursquare.com/docs/resources/categories):

food_category = '4d4b7105d754a06374d81259' # 'Root' category for all food-related venues

indian_restaurant_categories = ['4bf58dd8d48988d10f941735','54135bf5e4b08f3d2429dfe5','54135bf5e4b08f3d2429dff3',
                                 '54135bf5e4b08f3d2429dff5','54135bf5e4b08f3d2429dfe2','54135bf5e4b08f3d2429dff2',
                                 '54135bf5e4b08f3d2429dfe1','54135bf5e4b08f3d2429dfe3','54135bf5e4b08f3d2429dfe8',
                                 '54135bf5e4b08f3d2429dfe9','54135bf5e4b08f3d2429dfe6','54135bf5e4b08f3d2429dfdf',
                                 '54135bf5e4b08f3d2429dfe4','54135bf5e4b08f3d2429dfe7','54135bf5e4b08f3d2429dfea',
                                 '54135bf5e4b08f3d2429dfeb','54135bf5e4b08f3d2429dfed','54135bf5e4b08f3d2429dfee',
                                 '54135bf5e4b08f3d2429dff4','54135bf5e4b08f3d2429dfe0','54135bf5e4b08f3d2429dfdd',
                                 '54135bf5e4b08f3d2429dff6','54135bf5e4b08f3d2429dfef','54135bf5e4b08f3d2429dff0',
                                 '54135bf5e4b08f3d2429dff1','54135bf5e4b08f3d2429dfde','54135bf5e4b08f3d2429dfec']

def is_restaurant(categories, specific_filter=None):
    restaurant_words = ['restaurant', 'diner', 'taverna', 'steakhouse']
    restaurant = False
    specific = False
    for c in categories:
        category_name = c[0].lower()
        category_id = c[1]
        for r in restaurant_words:
            if r in category_name:
                restaurant = True
        if 'fast food' in category_name:
            restaurant = False
        if not(specific_filter is None) and (category_id in specific_filter):
            specific = True
            restaurant = True
    return restaurant, specific

def get_categories(categories):
    return [(cat['name'], cat['id']) for cat in categories]

def format_address(location):
    address = ', '.join(location['formattedAddress'])
    address = address.replace(', Deutschland', '')
    address = address.replace(', Germany', '')
    return address

def get_venues_near_location(lat, lon, category, client_id, client_secret, radius=500, limit=100):
    version = '20180724'
    url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&categoryId={}&radius={}&limit={}'.format(
        client_id, client_secret, version, lat, lon, category, radius, limit)
    try:
        results = requests.get(url).json()['response']['groups'][0]['items']
        venues = [(item['venue']['id'],
                   item['venue']['name'],
                   get_categories(item['venue']['categories']),
                   (item['venue']['location']['lat'], item['venue']['location']['lng']),
                   format_address(item['venue']['location']),
                   item['venue']['location']['distance']) for item in results]        
    except:
        venues = []
    return venues

In [14]:
# Let's now go over our neighborhood locations and get nearby restaurants; we'll also maintain a dictionary of all found restaurants and all found italian restaurants

import pickle

def get_restaurants(lats, lons):
    restaurants = {}
    indian_restaurants = {}
    location_restaurants = []

    print('Obtaining venues around candidate locations:', end='')
    for lat, lon in zip(lats, lons):
        # Using radius=350 to meke sure we have overlaps/full coverage so we don't miss any restaurant (we're using dictionaries to remove any duplicates resulting from area overlaps)
        venues = get_venues_near_location(lat, lon, food_category, foursquare_client_id, foursquare_client_secret, radius=350, limit=100)
        area_restaurants = []
        for venue in venues:
            venue_id = venue[0]
            venue_name = venue[1]
            venue_categories = venue[2]
            venue_latlon = venue[3]
            venue_address = venue[4]
            venue_distance = venue[5]
            is_res, is_indian = is_restaurant(venue_categories, specific_filter=indian_restaurant_categories)
            if is_res:
                x, y = lonlat_to_xy(venue_latlon[1], venue_latlon[0])
                restaurant = (venue_id, venue_name, venue_latlon[0], venue_latlon[1], venue_address, venue_distance, is_indian, x, y)
                if venue_distance<=300:
                    area_restaurants.append(restaurant)
                restaurants[venue_id] = restaurant
                if is_indian:
                    indian_restaurants[venue_id] = restaurant
        location_restaurants.append(area_restaurants)
        print(' .', end='')
    print(' done.')
    return restaurants, indian_restaurants, location_restaurants

# Try to load from local file system in case we did this before
restaurants = {}
indian_restaurants = {}
location_restaurants = []
loaded = False
try:
    with open('restaurants_350.pkl', 'rb') as f:
        restaurants = pickle.load(f)
    with open('italian_restaurants_350.pkl', 'rb') as f:
        italian_restaurants = pickle.load(f)
    with open('location_restaurants_350.pkl', 'rb') as f:
        location_restaurants = pickle.load(f)
    print('Restaurant data loaded.')
    loaded = True
except:
    pass

# If load failed use the Foursquare API to get the data
if not loaded:
    restaurants, indian_restaurants, location_restaurants = get_restaurants(latitudes, longitudes)
    
    # Let's persists this in local file system
    with open('restaurants_350.pkl', 'wb') as f:
        pickle.dump(restaurants, f)
    with open('indian_restaurants_350.pkl', 'wb') as f:
        pickle.dump(indian_restaurants, f)
    with open('location_restaurants_350.pkl', 'wb') as f:
        pickle.dump(location_restaurants, f)
        

Obtaining venues around candidate locations:

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:12: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:12: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:12: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:12: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:12: DeprecationWarning: This function

 . . .

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:12: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:12: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:12: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:12: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1


 .

...

 . . . .

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:12: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:12: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:12: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:12: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:12: DeprecationWarning: This function

 . . . .

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:12: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1


 . . . . . . . . . . . done.


In [15]:
import numpy as np

print('Total number of restaurants:', len(restaurants))
print('Total number of Indian restaurants:', len(indian_restaurants))
print('Percentage of Indian restaurants: {:.2f}%'.format(len(indian_restaurants) / len(restaurants) * 100))
print('Average number of restaurants in neighborhood:', np.array([len(r) for r in location_restaurants]).mean())

Total number of restaurants: 1370
Total number of Indian restaurants: 48
Percentage of Indian restaurants: 3.50%
Average number of restaurants in neighborhood: 14.652173913043478


In [16]:
print('List of all restaurants')
print('-----------------------')
for r in list(restaurants.values())[:10]:
    print(r)
print('...')
print('Total:', len(restaurants))

List of all restaurants
-----------------------
('556fb3b4498e61f524e70951', 'Maiella', 40.747341, -73.957053, '4610 Center Blvd (46th Avenue), Long Island City, NY 11109, United States', 86, False, -5816249.569069549, 9863447.3374533)
('526707f211d20789b84aea4c', 'Blend On The Water', 40.748326708206264, -73.95675323615949, '45-40 Center Blvd (at 46th Ave), Long Island City, NY 11109, United States', 88, False, -5816081.397998284, 9863413.327809712)
('4b47eb5cf964a520f54326e3', 'SHI', 40.74601152624443, -73.95700753754905, '4720 Center Blvd (at 47th Ave.), Long Island City, NY 11109, United States', 223, False, -5816474.785778439, 9863435.222019922)
('5112f745e4b095315641c547', 'Shi To Go', 40.74593542611723, -73.9568564789826, '4720 Center Blvd, Long Island City, NY 11101, United States', 235, False, -5816487.14634616, 9863415.38759545)
('4eb9eeaaf5b9ea8ad7395868', "Skinny's Cantina", 40.745628811029306, -73.95704496427506, '4705 Center Blvd (at 47th Rd), Long Island City, NY 11109, 

In [17]:
print('List of Indian restaurants')
print('---------------------------')
for r in list(indian_restaurants.values())[:10]:
    print(r)
print('...')
print('Total:', len(indian_restaurants))

List of Indian restaurants
---------------------------
('4fe4fb50c2eee335e4fea69d', 'Moti Mahal Delux', 40.76209761945207, -73.96028513480461, '1149 1st Ave (63rd Street), New York, NY 10065, United States', 219, True, -5813759.883232656, 9863933.224629175)
('592e1064345cbe358bbd5521', 'Imli', 40.761649, -73.960122, '1136 1st Ave (63rd), New York, NY 10065, United States', 270, True, -5813835.331582924, 9863910.100140614)
('4f78befde4b0c8b499531ad2', 'Swagat', 40.76229987477035, -73.96018330400798, '1154 1st Ave, New York, NY 10065, United States', 201, True, -5813725.243370076, 9863921.050623458)
('547253e6498e45b94affb8d8', 'Rangoli Exquisite Indian Cuisine', 40.76924655664551, -73.95814435124595, '1393A 2nd Ave (Btwn E. 72nd & E. 73rd Streets), New York, NY 10021, United States', 304, True, -5812540.775423606, 9863690.930469254)
('4a5b660df964a52015bb1fe3', 'Amma', 40.75562, -73.968666, '246 E 51st St (btwn 2nd & 3rd Ave), New York, NY 10022, United States', 172, True, -5814887.4978

In [18]:
print('Restaurants around location')
print('---------------------------')
for i in range(100, 110):
    rs = location_restaurants[i][:8]
    names = ', '.join([r[1] for r in rs])
    print('Restaurants around location {}: {}'.format(i+1, names))

Restaurants around location
---------------------------
Restaurants around location 101: Taboonette, Strip House, Strip House Speakeasy, Hu Kitchen, Da Andrea Restaurant, Babu Ji, Luke's Lobster Union Square, Tocqueville Restaurant
Restaurants around location 102: HALL, Sushi By Bou, Boqueria, da Umberto, OOTOYA 大戸屋 (OOTOYA), CAVA, Portale, Yakiniku Futago
Restaurants around location 103: CAVA, Momoya, Bite, Johny's Luncheonette, by CHLOE., Chop Shop II, Taïm, Tacombi Café El Presidente
Restaurants around location 104: Hanamizuki Cafe, Johny's Luncheonette, Maison Kayser, Piggyback, OCabanon, B & B Restaurant Corp, Milanes Spanish Restaurant, The Wilson
Restaurants around location 105: Panda Express, Penn Sushi, Dig, Harmony Istanbul, OCabanon, Versa, Hooters, Riko Peruvian Cuisine
Restaurants around location 106: Sergimmo Salumeria, Farida, Chirp, Tavola, Casa Nonna, Aiyara Thai, Go! Go! Curry!, Uncle Jack's Steakhouse
Restaurants around location 107: Farida, Sea Breeze Fish Market, C

Let's now see all the collected restaurants in our area of interest on map, and let's also show Indian restaurants in different color.

In [19]:
map_NYC = folium.Map(location=Times_Square, zoom_start=13)
folium.Marker(Times_Square, popup='Times Square').add_to(map_NYC)
for res in restaurants.values():
    lat = res[2]; lon = res[3]
    is_indian = res[6]
    color = 'red' if is_indian else 'blue'
    folium.CircleMarker([lat, lon], radius=3, color=color, fill=True, fill_color=color, fill_opacity=1).add_to(map_NYC)
map_NYC

Looking good. So now we have all the restaurants in area within few kilometers from Times Square, and we know which ones are Indian restaurants! We also know which restaurants exactly are in vicinity of every neighborhood candidate center.

This concludes the data gathering phase - we're now ready to use this data for analysis to produce the report on optimal locations for a new Indian restaurant!

## Methodology <a name="methodology"></a>

In this project we will direct our efforts on detecting areas of Manhattan that have low restaurant density, particularly those with low number of Indian restaurants. We will limit our analysis to area ~4km around Times Square.

In first step we have collected the required **data: location and type (category) of every restaurant within 4km from Times Square** We have also **identified Indian restaurants** (according to Foursquare categorization).

Second step in our analysis will be calculation and exploration of '**restaurant density**' across different areas of Manhattan - we will use **heatmaps** to identify a few promising areas close to center with low number of restaurants in general (*and* no Indian restaurants in vicinity) and focus our attention on those areas.

In third and final step we will focus on most promising areas and within those create **clusters of locations that meet some basic requirements** established in discussion with stakeholders: we will take into consideration locations with **no more than ten restaurants in radius of 250 meters**, and we want locations **without Indian restaurants in radius of 400 meters**. We will present map of all such locations but also create clusters (using **k-means clustering**) of those locations to identify general zones / neighborhoods / addresses which should be a starting point for final 'street level' exploration and search for optimal venue location by stakeholders.

## Analysis <a name="analysis"></a>

Let's perform some basic explanatory data analysis and derive some additional info from our raw data. First let's count the **number of restaurants in every area candidate**:

In [20]:
location_restaurants_count = [len(res) for res in location_restaurants]

df_locations['Restaurants in area'] = location_restaurants_count

print('Average number of restaurants in every area with radius=300m:', np.array(location_restaurants_count).mean())

df_locations.head(10)

Average number of restaurants in every area with radius=300m: 14.652173913043478


,Address,Latitude,Longitude,X,Y,Distance from center,Restaurants in area
0,"Long Island City, Center Blvd &, 46th Ave, New...",40.747956,-73.957686,-5.816148e+06,9.863532e+06,3973.663297,5
1,"Southpoint Park, Roosevelt Island, NY 10044",40.751493,-73.957557,-5.815548e+06,9.863532e+06,3772.267223,0
2,"200 W Loop Rd, New York, NY 10044",40.755030,-73.957428,-5.814948e+06,9.863532e+06,3659.234893,0
3,"31 Ed Koch Queensboro Bridge Path, New York, N...",40.758567,-73.957299,-5.814348e+06,9.863532e+06,3642.801120,3
4,"1194 York Ave, New York, NY 10065",40.762105,-73.957170,-5.813748e+06,9.863532e+06,3724.244890,24
5,"404 E 69th St, New York, NY 10021",40.765642,-73.957041,-5.813148e+06,9.863532e+06,3897.435054,7
6,"1-01-1-02 54th Ave, Long Island City, NY 11101",40.742736,-73.961906,-5.817048e+06,9.864051e+06,3996.248241,2
7,"4749 Center Blvd, Long Island City, NY 11109",40.746272,-73.961777,-5.816448e+06,9.864051e+06,3651.027253,0
8,"E Rd, New York, NY 10044",40.749809,-73.961649,-5.815848e+06,9.864051e+06,3377.869151,0
9,"1025 E 51st Street Pedestrian Crossing, New Yo...",40.753346,-73.961520,-5.815248e+06,9.864051e+06,3195.309062,0


OK, now let's calculate the **distance to nearest Indian restaurant from every area candidate center** (not only those within 300m - we want distance to closest one, regardless of how distant it is).

In [21]:
distances_to_indian_restaurant = []

for area_x, area_y in zip(xs, ys):
    min_distance = 10000
    for res in indian_restaurants.values():
        res_x = res[7]
        res_y = res[8]
        d = calc_xy_distance(area_x, area_y, res_x, res_y)
        if d<min_distance:
            min_distance = d
    distances_to_indian_restaurant.append(min_distance)

df_locations['Distance to Indian restaurant'] = distances_to_indian_restaurant

In [22]:
df_locations.head(10)

,Address,Latitude,Longitude,X,Y,Distance from center,Restaurants in area,Distance to Indian restaurant
0,"Long Island City, Center Blvd &, 46th Ave, New...",40.747956,-73.957686,-5.816148e+06,9.863532e+06,3973.663297,5,1921.973323
1,"Southpoint Park, Roosevelt Island, NY 10044",40.751493,-73.957557,-5.815548e+06,9.863532e+06,3772.267223,0,1594.331369
2,"200 W Loop Rd, New York, NY 10044",40.755030,-73.957428,-5.814948e+06,9.863532e+06,3659.234893,0,1174.813837
3,"31 Ed Koch Queensboro Bridge Path, New York, N...",40.758567,-73.957299,-5.814348e+06,9.863532e+06,3642.801120,3,636.788149
4,"1194 York Ave, New York, NY 10065",40.762105,-73.957170,-5.813748e+06,9.863532e+06,3724.244890,24,388.343587
5,"404 E 69th St, New York, NY 10021",40.765642,-73.957041,-5.813148e+06,9.863532e+06,3897.435054,7,627.314633
6,"1-01-1-02 54th Ave, Long Island City, NY 11101",40.742736,-73.961906,-5.817048e+06,9.864051e+06,3996.248241,2,2179.586427
7,"4749 Center Blvd, Long Island City, NY 11109",40.746272,-73.961777,-5.816448e+06,9.864051e+06,3651.027253,0,1817.090613
8,"E Rd, New York, NY 10044",40.749809,-73.961649,-5.815848e+06,9.864051e+06,3377.869151,0,1337.807668
9,"1025 E 51st Street Pedestrian Crossing, New Yo...",40.753346,-73.961520,-5.815248e+06,9.864051e+06,3195.309062,0,998.819511


In [23]:
print('Average distance to closest Indian restaurant from each area center:', df_locations['Distance to Indian restaurant'].mean())

Average distance to closest Indian restaurant from each area center: 900.2108274133141


OK, so **on average Indian restaurant can be found within ~1km** from every area center candidate. That's close, so we need to filter our areas carefully!

Let's crete a map showing **heatmap / density of restaurants** and try to extract some meaningfull info from that. Also, let's show neighbourhoods of Manhattan on our map and a few circles indicating distance of 1km and 2km from Times Square.

In [24]:
restaurant_latlons = [[res[2], res[3]] for res in restaurants.values()]

indian_latlons = [[res[2], res[3]] for res in indian_restaurants.values()]

In [25]:
from folium import plugins
from folium.plugins import HeatMap

map_NYC = folium.Map(location=Times_Square, zoom_start=13)
folium.TileLayer('cartodbpositron').add_to(map_NYC) #cartodbpositron cartodbdark_matter
HeatMap(restaurant_latlons).add_to(map_NYC)
folium.Marker(Times_Square).add_to(map_NYC)
folium.Circle(Times_Square, radius=1000, fill=False, color='white').add_to(map_NYC)
folium.Circle(Times_Square, radius=2000, fill=False, color='white').add_to(map_NYC)
map_NYC

Looks like a restaurant density is quite good around Times Square and in general.. 

Let's create another heatmap map showing **heatmap/density of Indian restaurants** only.

In [26]:
map_NYC = folium.Map(location=Times_Square, zoom_start=13)
folium.TileLayer('cartodbpositron').add_to(map_NYC) #cartodbpositron cartodbdark_matter
HeatMap(indian_latlons).add_to(map_NYC)
folium.Marker(Times_Square).add_to(map_NYC)
folium.Circle(Times_Square, radius=1000, fill=False, color='white').add_to(map_NYC)
folium.Circle(Times_Square, radius=2000, fill=False, color='white').add_to(map_NYC)
map_NYC

This map is not so 'hot' (Indian restaurants represent a subset of ~3.5% of all restaurants in Manhattan). The Indian restaurants density is low pretty much everyhere except west of Times Square. And in 1.5 km around times square the density is pretty low.

Based on this, we will focus our analysis on areas around Times Square within 1.5 km radius.

In [27]:
roi_x_min = Times_Square_x
roi_y_max = Times_Square_y
roi_width = 1500
roi_height = 1500
roi_center_x = roi_x_min
roi_center_y = roi_y_max
roi_center_lon, roi_center_lat = xy_to_lonlat(roi_center_x, roi_center_y)
roi_center = [roi_center_lat, roi_center_lon]

map_NY = folium.Map(location=roi_center, zoom_start=14)
HeatMap(restaurant_latlons).add_to(map_NY)
folium.Marker(Times_Square).add_to(map_NY)
folium.Circle(roi_center, radius=1500, color='white', fill=True, fill_opacity=0.4).add_to(map_NY)
map_NY

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:18: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1


This covers all the pockets of low restaurant density in the west of Times Square.

Let's also create new, more dense grid of location candidates restricted to our new region of interest (let's make our location candidates 100m appart).

In [28]:
k = math.sqrt(3) / 2 # Vertical offset for hexagonal grid cells
x_step = 100
y_step = 100 * k 
roi_y_min = roi_center_y

roi_latitudes = []
roi_longitudes = []
roi_xs = []
roi_ys = []
for i in range(0, int(51/k)):
    y = roi_y_min + i * y_step
    x_offset = 50 if i%2==0 else 0
    for j in range(0, 51):
        x = roi_x_min + j * x_step + x_offset
        d = calc_xy_distance(roi_center_x, roi_center_y, x, y)
        if (d <= 1501):
            lon, lat = xy_to_lonlat(x, y)
            roi_latitudes.append(lat)
            roi_longitudes.append(lon)
            roi_xs.append(x)
            roi_ys.append(y)

print(len(roi_latitudes), 'candidate neighborhood centers generated.')

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:18: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:18: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:18: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:18: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:18: DeprecationWarning: This function

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:18: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:18: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:18: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:18: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:18: DeprecationWarning: This function

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:18: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:18: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:18: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:18: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:18: DeprecationWarning: This function

215 candidate neighborhood centers generated.


/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:18: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:18: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:18: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:18: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:18: DeprecationWarning: This function

OK. Now let's calculate two most important things for each location candidate: **number of restaurants in vicinity** (we'll use radius of **250 meters**) and **distance to closest Indian restaurant**.

In [29]:
def count_restaurants_nearby(x, y, restaurants, radius=250):    
    count = 0
    for res in restaurants.values():
        res_x = res[7]; res_y = res[8]
        d = calc_xy_distance(x, y, res_x, res_y)
        if d<=radius:
            count += 1
    return count

def find_nearest_restaurant(x, y, restaurants):
    d_min = 100000
    for res in restaurants.values():
        res_x = res[7]; res_y = res[8]
        d = calc_xy_distance(x, y, res_x, res_y)
        if d<=d_min:
            d_min = d
    return d_min

roi_restaurant_counts = []
roi_indian_distances = []

print('Generating data on location candidates... ', end='')
for x, y in zip(roi_xs, roi_ys):
    count = count_restaurants_nearby(x, y, restaurants, radius=250)
    roi_restaurant_counts.append(count)
    distance = find_nearest_restaurant(x, y, indian_restaurants)
    roi_indian_distances.append(distance)
print('done.')


Generating data on location candidates... done.


In [30]:
# Let's put this into dataframe
df_roi_locations = pd.DataFrame({'Latitude':roi_latitudes,
                                 'Longitude':roi_longitudes,
                                 'X':roi_xs,
                                 'Y':roi_ys,
                                 'Restaurants nearby':roi_restaurant_counts,
                                 'Distance to Indian restaurant':roi_indian_distances})

df_roi_locations.head(10)

,Latitude,Longitude,X,Y,Restaurants nearby,Distance to Indian restaurant
0,40.758269,-73.985532,-5.814498e+06,9.867169e+06,13,205.816072
1,40.758859,-73.985511,-5.814398e+06,9.867169e+06,9,231.600141
2,40.759449,-73.985490,-5.814298e+06,9.867169e+06,7,291.405207
3,40.760038,-73.985469,-5.814198e+06,9.867169e+06,9,369.046561
4,40.760628,-73.985448,-5.814098e+06,9.867169e+06,11,455.492847
5,40.761217,-73.985428,-5.813998e+06,9.867169e+06,8,546.582202
6,40.761807,-73.985407,-5.813898e+06,9.867169e+06,11,640.336219
7,40.762397,-73.985386,-5.813798e+06,9.867169e+06,12,634.979371
8,40.762986,-73.985365,-5.813698e+06,9.867169e+06,11,543.432097
9,40.763576,-73.985344,-5.813598e+06,9.867169e+06,10,455.453715


OK. Let us now **filter** those locations: we're interested only in **locations with no more than ten restaurants in radius of 250 meters**, and **no Indian restaurants in radius of 400 meters**.

In [31]:
good_res_count = np.array((df_roi_locations['Restaurants nearby']<=10))
print('Locations with no more than ten restaurants nearby:', good_res_count.sum())

good_ita_distance = np.array(df_roi_locations['Distance to Indian restaurant']>=400)
print('Locations with no Indian restaurants within 400m:', good_ita_distance.sum())

good_locations = np.logical_and(good_res_count, good_ita_distance)
print('Locations with both conditions met:', good_locations.sum())

df_good_locations = df_roi_locations[good_locations]

Locations with no more than ten restaurants nearby: 91
Locations with no Indian restaurants within 400m: 154
Locations with both conditions met: 60


Let's see how this looks on a map.

In [32]:
good_latitudes = df_good_locations['Latitude'].values
good_longitudes = df_good_locations['Longitude'].values

good_locations = [[lat, lon] for lat, lon in zip(good_latitudes, good_longitudes)]

map_NYC = folium.Map(location=roi_center, zoom_start=14)
folium.TileLayer('cartodbpositron').add_to(map_NYC)
HeatMap(restaurant_latlons).add_to(map_NYC)
folium.Circle(roi_center, radius=1500, color='white', fill=True, fill_opacity=0.6).add_to(map_NYC)
folium.Marker(Times_Square).add_to(map_NYC)
for lat, lon in zip(good_latitudes, good_longitudes):
    folium.CircleMarker([lat, lon], radius=2, color='blue', fill=True, fill_color='blue', fill_opacity=1).add_to(map_NYC) 
map_NYC

Looking good. We now have a bunch of locations fairly close to Times Square, and we know that each of those locations has no more than ten restaurants in radius of 250m, and no Indian restaurant closer than 400m. Any of those locations is a potential candidate for a new Indian restaurant, at least based on nearby competition.

Let's now show those good locations in a form of heatmap:

In [33]:
map_NYC = folium.Map(location=roi_center, zoom_start=14)
HeatMap(good_locations, radius=25).add_to(map_NYC)
folium.Marker(Times_Square).add_to(map_NYC)
for lat, lon in zip(good_latitudes, good_longitudes):
    folium.CircleMarker([lat, lon], radius=2, color='blue', fill=True, fill_color='blue', fill_opacity=1).add_to(map_NYC)
map_NYC

Looking good. What we have now is a clear indication of zones with low number of restaurants in vicinity, and *no* Indian restaurants at all nearby.

Let us now **cluster** those locations to create **centers of zones containing good locations**. Those zones, their centers and addresses will be the final result of our analysis. 

In [34]:
from sklearn.cluster import KMeans

number_of_clusters = 5

good_xys = df_good_locations[['X', 'Y']].values
kmeans = KMeans(n_clusters=number_of_clusters, random_state=0).fit(good_xys)

cluster_centers = [xy_to_lonlat(cc[0], cc[1]) for cc in kmeans.cluster_centers_]

map_NYC = folium.Map(location=roi_center, zoom_start=14)
folium.Marker(Times_Square, popup='Times Square').add_to(map_NYC)
#folium.TileLayer('cartodbpositron').add_to(newyork_data)
HeatMap(restaurant_latlons).add_to(map_NYC)
folium.Circle(roi_center, radius=1500, color='white', fill=True, fill_opacity=0.4).add_to(map_NYC)
folium.Marker(Times_Square).add_to(map_NYC)
for lon, lat in cluster_centers:
    folium.Circle([lat, lon], radius=200, color='green', fill=True, fill_opacity=0.25).add_to(map_NYC) 
for lat, lon in zip(good_latitudes, good_longitudes):
    folium.CircleMarker([lat, lon], radius=2, color='blue', fill=True, fill_color='blue', fill_opacity=1).add_to(map_NYC)
map_NYC

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:18: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:18: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:18: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:18: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:18: DeprecationWarning: This function

Not bad - our clusters represent groupings of most of the candidate locations and cluster centers are placed nicely in the middle of the zones 'rich' with location candidates.

Addresses of those cluster centers will be a good starting point for exploring the neighborhoods to find the best possible location based on neighborhood specifics.

Let's see those zones on a city map without heatmap, using shaded areas to indicate our clusters:

In [35]:
map_NYC = folium.Map(location=roi_center, zoom_start=14)
folium.Marker(Times_Square).add_to(map_NYC)
for lat, lon in zip(good_latitudes, good_longitudes):
    folium.Circle([lat, lon], radius=250, color='#00000000', fill=True, fill_color='#0066ff', fill_opacity=0.07).add_to(map_NYC)
for lat, lon in zip(good_latitudes, good_longitudes):
    folium.CircleMarker([lat, lon], radius=2, color='blue', fill=True, fill_color='blue', fill_opacity=1).add_to(map_NYC)
for lon, lat in cluster_centers:
    folium.Circle([lat, lon], radius=200, color='green', fill=False).add_to(map_NYC) 
map_NYC

Finaly, let's **reverse geocode those candidate area centers to get the addresses** which can be presented to stakeholders.

In [36]:
candidate_area_addresses = []
print('==============================================================')
print('Addresses of centers of areas recommended for further analysis')
print('==============================================================\n')
for lon, lat in cluster_centers:
    addr = get_address(google_api_key, lat, lon).replace(', USA', '')
    candidate_area_addresses.append(addr)    
    x, y = lonlat_to_xy(lon, lat)
    d = calc_xy_distance(x, y, Times_Square_x, Times_Square_y)
    print('{}{} => {:.1f}km from Times Square'.format(addr, ' '*(50-len(addr)), d/1000))
    

Addresses of centers of areas recommended for further analysis

Parking, 560 10th Ave, New York, NY 10036          => 1.3km from Times Square


/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:12: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:12: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1


825 8th Ave, New York, NY 10019                    => 0.7km from Times Square
711 8th Ave, New York, NY 10036                    => 0.4km from Times Square


/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:12: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:12: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1


444 W 52nd St, New York, NY 10019                  => 1.4km from Times Square
605 10th Ave, New York, NY 10036                   => 1.3km from Times Square


/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:12: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1


This concludes our analysis. We have created 5 addresses representing centers of zones containing locations with low density of restaurants and no Indian restaurants nearby, all zones being fairly close to city center (all less than 1km from Times Square, and about half of those less than 500m from Times Square). Although zones are shown on map with a radius of ~500 meters (green circles), their shape is actually very irregular and their centers/addresses should be considered only as a starting point for exploring area neighborhoods in search for potential restaurant locations.

In [37]:
map_NYC = folium.Map(location=roi_center, zoom_start=14)
folium.Circle(Times_Square, radius=50, color='red', fill=True, fill_color='red', fill_opacity=1).add_to(map_NYC)
for lonlat, addr in zip(cluster_centers, candidate_area_addresses):
    folium.Marker([lonlat[1], lonlat[0]], popup=addr).add_to(map_NYC) 
for lat, lon in zip(good_latitudes, good_longitudes):
    folium.Circle([lat, lon], radius=250, color='#0000ff00', fill=True, fill_color='#0066ff', fill_opacity=0.05).add_to(map_NYC)
map_NYC

## Results and Discussion <a name="results"></a>

Our analysis shows that there is a great number of restaurants in Manhattan (~1400 in our initial area of interest which was 4X4 km around Times Square), there are pockets of low Indian restaurant density fairly close to city center. So we focused our attention to these areas which are particularly to the west of Times Square.

After directing our attention to this more narrow area of interest (covering approx. 1.5X1.5km west from Times Square) we first created a dense grid of location candidates (spaced 100m appart); those locations were then filtered so that those with more than ten restaurants in radius of 250m and those with an Indian restaurant closer than 400m were removed.

Those location candidates were then clustered to create zones of interest which contain greatest number of location candidates. Addresses of centers of those zones were also generated using reverse geocoding to be used as markers/starting points for more detailed local analysis based on other factors.

Result of all this is 5 zones containing largest number of potential new restaurant locations based on number of and distance to existing venues - both restaurants in general and Indian restaurants particularly. This, of course, does not imply that those zones are actually optimal locations for a new restaurant! Purpose of this analysis was to only provide info on areas close to Times Square but not crowded with existing restaurants (particularly Indian) - it is entirely possible that there is a very good reason for small number of restaurants in any of those areas, reasons which would make them unsuitable for a new restaurant regardless of lack of competition in the area. Recommended zones should therefore be considered only as a starting point for more detailed analysis which could eventually result in location which has not only no nearby competition but also other factors taken into account and all other relevant conditions met.

## Conclusion <a name="conclusion"></a>

Purpose of this project was to identify Manhattan areas close to Times Square (heart of New York City) with low number of Indian restaurants in order to aid stakeholders in narrowing down the search for optimal location for a new Indian restaurant. By calculating restaurant density distribution from Foursquare data we have first identified general areas that justify further analysis (west of Times Square), and then generated extensive collection of locations which satisfy some basic requirements regarding existing nearby restaurants. Clustering of those locations was then performed in order to create major zones of interest (containing greatest number of potential locations) and addresses of those zone centers were created to be used as starting points for final exploration by stakeholders.

Final decission on optimal restaurant location will be made by stakeholders based on specific characteristics of neighborhoods and locations in every recommended zone, taking into consideration additional factors like attractiveness of each location (proximity to park or water), levels of noise / proximity to major roads, real estate availability, prices, social and economic dynamics of every neighborhood etc.